In [2]:
import pandas as pd
import numpy as np

In [3]:
import sys
sys.path.append('../../code')

from utils import binarize_log_data, remove_ambiguous_row, clean_continuous

In [4]:
slice = pd.read_csv('../../data/raw/chembl30_slice.csv')

/tmp/ipykernel_3880917/4181517134.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  slice = pd.read_csv('../../data/raw/chembl30_slice.csv')


In [5]:
slice['standard_value'] = slice['standard_value'].map(lambda value: -np.log10((max(value, 0) + 1)) + 9)
slice = slice[slice['standard_type'] == 'Ki']
drd = slice[slice['gene_symbol'] == 'DRD2']
drd

,Unnamed: 0,compound_chembl_id,canonical_smiles,standard_type,standard_value,standard_units,standard_relation,assay_id,target_chembl_id,activity_comment,data_validity_comment,organism,pref_name,gene_symbol
1673,43358,CHEMBL6437,CN1CCN2c3ccccc3Cc3ccccc3C2C1,Ki,5.657972,nM,=,303146,CHEMBL217,NaN,NaN,Homo sapiens,Dopamine D2 receptor,DRD2
1976,48837,CHEMBL269004,CCCN(CCC)[C@H]1CCc2c(O)cccc2C1,Ki,7.223299,nM,=,851588,CHEMBL339,NaN,NaN,Rattus norvegicus,Dopamine D2 receptor,DRD2
1981,48844,CHEMBL269004,CCCN(CCC)[C@H]1CCc2c(O)cccc2C1,Ki,7.221849,nM,=,880031,CHEMBL339,NaN,NaN,Rattus norvegicus,Dopamine D2 receptor,DRD2
1983,48846,CHEMBL269004,CCCN(CCC)[C@H]1CCc2c(O)cccc2C1,Ki,7.223299,nM,=,961547,CHEMBL339,NaN,NaN,Rattus norvegicus,Dopamine D2 receptor,DRD2
1985,48848,CHEMBL269004,CCCN(CCC)[C@H]1CCc2c(O)cccc2C1,Ki,6.812479,nM,=,1541731,CHEMBL339,NaN,NaN,Rattus norvegicus,Dopamine D2 receptor,DRD2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2669207,16430106,CHEMBL4878501,O=C(NCCCCN1CCO[C@@H]2c3ccccc3OC[C@H]21)c1ccc2c...,Ki,6.140261,nM,=,2116072,CHEMBL217,NaN,NaN,Homo sapiens,Dopamine D2 receptor,DRD2
2669271,16430350,CHEMBL4878577,O=C(NCCCCN1CCO[C@@H]2c3ccccc3OC[C@H]21)c1cccc(...,Ki,6.177832,nM,=,2116072,CHEMBL217,NaN,NaN,Homo sapiens,Dopamine D2 receptor,DRD2
2669278,16430368,CHEMBL4878587,COc1cc2c(cc1OC)CN(Cc1ccccc1CNC(=O)c1ccc(C#N)cc...,Ki,7.292430,nM,=,2113158,CHEMBL217,NaN,NaN,Homo sapiens,Dopamine D2 receptor,DRD2
2669412,16431038,CHEMBL4878826,O=C(NCCCCN1CCO[C@@H]2c3ccccc3OC[C@H]21)c1ccc(C...,Ki,6.324222,nM,=,2116072,CHEMBL217,NaN,NaN,Homo sapiens,Dopamine D2 receptor,DRD2


# Prepare DRD2-Hi

In [7]:
drd_binary = binarize_log_data(drd, threshold=6)
drd_binary = remove_ambiguous_row(drd_binary)
drd_hi = pd.DataFrame({
    'smiles': drd_binary.index.to_list(),
    'value': drd_binary['label'] > 0.5
}).reset_index(drop=True)
drd_hi

,smiles,value
0,Brc1ccc(-[n+]2cc[n+](Cc3ccccc3)cc2)c2cc[nH]c12,True
1,Brc1ccc(CNCCN2CCN(Cc3cc4ccccc4[nH]3)CC2)cc1,False
2,Brc1ccc(N2CCN(Cc3ccccc3)CC2)c2cc[nH]c12,True
3,Brc1ccc(NCCN2CCN(CCc3c[nH]c4ccccc34)CC2)cc1,True
4,Brc1ccc(NCCN2CCN(Cc3cc4ccccc4[nH]3)CC2)cc1,True
...,...,...
6262,c1cnc(N2CCN(CCCOc3ccc(-c4nc5ccccc5o4)cc3)CC2)nc1,True
6263,c1cnc(N2CCN(CCCSc3nc4ccccc4s3)CC2)nc1,True
6264,c1cnc(N2CCN(Cc3c[nH]c4ncccc34)CC2)nc1,False
6265,c1cncc(CN[C@H]2C3C4CC5C6C4CC3C6C52)c1,False


In [10]:
drd_hi.to_csv('../../data/raw/drd2_hi.csv')

# Prepare DRD2-Lo

In [6]:
drd_continuous = clean_continuous(drd)
drd_lo = pd.DataFrame({
    'smiles': drd_continuous['canonical_smiles'],
    'value': drd_continuous['standard_value']
}).reset_index(drop=True)
drd_lo

/data/steshin/gero_benchmark/notebooks/data/../../code/utils.py:85: FutureWarning: Dropping invalid columns in DataFrameGroupBy.min is deprecated. In a future version, a TypeError will be raised. Before calling .min, select only columns which should be valid for the function.
  min_values = group.min()
/data/steshin/gero_benchmark/notebooks/data/../../code/utils.py:86: FutureWarning: Dropping invalid columns in DataFrameGroupBy.max is deprecated. In a future version, a TypeError will be raised. Before calling .max, select only columns which should be valid for the function.
  max_values = group.max()
/data/steshin/gero_benchmark/notebooks/data/../../code/utils.py:90: FutureWarning: The default value of numeric_only in DataFrameGroupBy.median is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  continuous_clean = group.median()


,smiles,value
0,Brc1ccc(-[n+]2cc[n+](Cc3ccccc3)cc2)c2cc[nH]c12,7.717691
1,Brc1ccc(CNCCN2CCN(Cc3cc4ccccc4[nH]3)CC2)cc1,5.283913
2,Brc1ccc(N2CCN(Cc3ccccc3)CC2)c2cc[nH]c12,7.437357
3,Brc1ccc(NCCN2CCN(CCc3c[nH]c4ccccc34)CC2)cc1,7.288705
4,Brc1ccc(NCCN2CCN(Cc3cc4ccccc4[nH]3)CC2)cc1,6.035740
...,...,...
5414,c1cnc(N2CCN(CCCOc3ccc(-c4nc5ccccc5[nH]4)cc3)CC...,6.568636
5415,c1cnc(N2CCN(CCCOc3ccc(-c4nc5ccccc5o4)cc3)CC2)nc1,6.701147
5416,c1cnc(N2CCN(CCCSc3nc4ccccc4s3)CC2)nc1,6.273273
5417,c1cnc(N2CCN(Cc3c[nH]c4ncccc34)CC2)nc1,5.931443


In [7]:
drd_lo.to_csv('../../data/raw/drd2_lo.csv')